In [46]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [47]:
#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [48]:
#HMP dataset
text_file = "/home/blooh/ersp/merged_bmi_mapping_final__original_study_HMP__.txt"
biom_file = "/home/blooh/ersp/filtered_otu_table__original_study_HMP__.biom"

In [49]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [50]:
m.columns

Index([u'BarcodeSequence', u'LinkerPrimerSequence', u'BMI',
       u'bmi_group_binned', u'bmi_group_coded', u'original_study',
       u'combined_study_bmi_group', u'PCR_PRIMERS', u'TARGET_SUBFRAGMENT',
       u'AGE', u'ELEVATION', u'LONGITUDE', u'COUNTRY', u'SEQUENCING_METH',
       u'SAMPLE_CENTER', u'Description_duplicate', u'ReversePrimer',
       u'COLLECTION_DATE', u'SEX', u'FAMILY_RELATIONSHIP_GG', u'STUDY_CENTER',
       u'EXPERIMENT_CENTER', u'bmi_group_amish', u'RUN_CENTER', u'LATITUDE',
       u'Description'],
      dtype='object')

In [51]:
m.index

Index([u'SRS011271.396367', u'SRS011410.396107', u'SRS011415.394388',
       u'SRS011529.396185', u'SRS011586.393765', u'SRS011621.394454',
       u'SRS012191.395151', u'SRS013177.401537', u'SRS013386.400409',
       u'SRS013543.394407',
       ...
       u'SRS049157.403739', u'SRS049959.401022', u'SRS056620.394943',
       u'SRS057447.396161', u'SRS063068.403097', u'SRS063524.397569',
       u'SRS063827.396534', u'SRS065263.395822', u'SRS065466.395776',
       u'SRS065725.395071'],
      dtype='object', name=u'#SampleID', length=208)

In [52]:
b= biom.load_table(biom_file)

In [53]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
i_2 = m[m.bmi_group_coded != "Overweight"]

i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0

In [54]:
# prints 4th column
y = m.iloc[:, 4]
print(y)
print(y.size)

#SampleID
SRS011271.396367        Normal
SRS011410.396107        Normal
SRS011415.394388        Normal
SRS011529.396185        Normal
SRS011586.393765        Normal
SRS011621.394454        Normal
SRS012191.395151        Normal
SRS013177.401537        Normal
SRS013386.400409        Normal
SRS013543.394407        Normal
SRS013687.400715        Normal
SRS013801.398248        Normal
SRS014345.393344        Normal
SRS014572.396411        Normal
SRS014885.394865        Normal
SRS014923.394014        Normal
SRS015247.394888        Normal
SRS015389.394987        Normal
SRS015599.394288        Normal
SRS015724.395613        Normal
SRS015782.394538        Normal
SRS015911.396089        Normal
SRS015960.395111        Normal
SRS016095.395879        Normal
SRS016267.395183        Normal
SRS016335.395358        Normal
SRS016400.399670        Normal
SRS016437.400001        Normal
SRS016517.399341        Normal
SRS016585.401158        Normal
                       ...    
SRS020926.402304    Overweigh

In [55]:
writer = pd.ExcelWriter('biom_original_1.xlsx', engine='xlsxwriter')

In [56]:
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
print(p)

                  304309  193008  190299  191393  193061  186596  174924  \
SRS057447.396161       0       0       0       0       0       0       0   
SRS013762.393921       0       0       0       0       0       0       0   
SRS011415.394388       0       0       0       0       0       0       0   
SRS052697.393711       0       0       0       0       0       0       0   
SRS052433.395864       0       0       0       0       0       0       0   
SRS048722.395742       0       0       0       0       0       0       2   
SRS019910.396375       0       0       0       0       0       0       0   
SRS015247.394888       0       0       0       0       0       0       2   
SRS014999.393232       0       0       0       0       0       0       0   
SRS018968.393467       0       0       0       0       0       0       2   
SRS022480.394346       0       0       0       0       0       0       0   
SRS012191.395151       0       0       0       0       0       0       1   
SRS063921.39

In [57]:
# Convert the dataframe to an XlsxWriter Excel object.
p.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [58]:
# load sorted file
sorted_m = pd.read_csv("/home/blooh/ersp/biom_original_1.csv", index_col=0)
print(sorted_m)

CParserError: Error tokenizing data. C error: Expected 8 fields in line 4, saw 16


In [ ]:
c = p.iloc[i, :]
print(c)

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
#10 fold cross-validation, 5 times repeated
#for i in range(5):
   # kfold = model_selection.KFold(n_splits=10, random_state=7)
    #model = RandomForestClassifier(n_estimators=1000)
    #results = model_selection.cross_val_score(model, p.fillna(0).as_matrix(), y, cv=kfold)
    #print(results.mean())

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
iter_range = range(1, 10)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    predicted = cross_val_predict(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10)
    #cross-validation AUC score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    #rForest_scores.append(scores.mean())
    #average_accuracy+=scores.mean()
    print(confusion_matrix(i_2.bmi_group_binned, predicted))

#print(rForest_scores)
#print(average_accuracy/(5.0))

In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    #cross-validation AUC score
    scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))